In [2]:
import os
import sys
import math
import numpy as np
import netCDF4 as nc
from matplotlib import pyplot as plt
from math import exp, sqrt, log
import time
import geopandas as gpd
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
if '..' not in sys.path:
    sys.path.append('..')
from importlib import reload
#reload(util_tools)
import util_tools
from util_tools.data_loader import data_processer

# 1. Data Processing

In [3]:
# define necessary parameters
target_var = 'DUEXTTAU'
file_path_g_06 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20060516-20070515.nc'
file_path_g_05 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20050516-20060515.nc'
file_path_m = r'C:\Users\96349\Documents\Downscale_data\MERRA2\MERRA2_aerosol_variables_over_MiddleEast_daily_20000516-20180515.nc'
file_path_ele = r'C:\Users\96349\Documents\Downscale_data\elevation\elevation_data.npy'
file_path_country = [r'C:\Users\96349\Documents\Downscale_data\Country_shape\AFG_adm/AFG_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\ARE_adm/ARE_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\IRQ_adm/IRQ_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\KWT_adm/KWT_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\QAT_adm/QAT_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\SAU_adm/SAU_adm0.shp']
n_lag = 15
n_pred = 1
task_dim = [5, 5]

In [4]:
#reload(util_tools.data_loader)
import util_tools.data_loader as data_loader

In [5]:
data_processor = data_loader.data_processer()
g_data, m_data, [G_lats, G_lons, M_lats, M_lons], ele_data = data_processor.load_data(target_var, file_path_g_05, file_path_g_06, file_path_m, file_path_ele, file_path_country)

In [6]:
m_data = data_processor.unify_m_data(g_data[:10], m_data, G_lats, G_lons, M_lats, M_lons)

In [7]:
match_m_data = m_data[range(1826, 1826+730), :, :]

In [8]:
g_data.shape

(730, 123, 207)

In [15]:
7%8

7

In [27]:
X_high, X_low, X_ele, X_other, Y = data_processor.flatten(g_data[:50, 60:, :100], match_m_data[:50, 60:, :100], ele_data[60:, :100],
                                                          [G_lats[60:], G_lons[:100]], list(range(50)), n_lag=n_lag, n_pred=n_pred, task_dim=task_dim, is_perm=True, 
                                                          return_Y=True)

In [18]:
def area_code(area):
    lat_id = area//7 + 1 if area%7 != 0 else area//7
    lon_id = area%7 if area%7 != 0 else 7
    print(area, lat_id, lon_id)
for i in range(1, 29):
    area_code(i)

1 1 1
2 1 2
3 1 3
4 1 4
5 1 5
6 1 6
7 1 7
8 2 1
9 2 2
10 2 3
11 2 4
12 2 5
13 2 6
14 2 7
15 3 1
16 3 2
17 3 3
18 3 4
19 3 5
20 3 6
21 3 7
22 4 1
23 4 2
24 4 3
25 4 4
26 4 5
27 4 6
28 4 7


# 2. GAN model test

In [28]:
def nnelu(input):
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

def mapping_to_target_range( x, target_min=0, target_max=1 ) :
    x02 = tf.keras.backend.tanh(x) + 1 # x in range(0,2)
    scale = ( target_max-target_min )/2.
    return  x02 * scale + target_min


high_input = tf.keras.Input(shape=(n_lag, task_dim[0], task_dim[1], 1))
x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), return_sequences=True, activation=tf.keras.layers.LeakyReLU())(high_input)
x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(x1)
x1 = tf.keras.layers.Flatten()(x1)

low_input = tf.keras.Input(shape=(n_lag, task_dim[0], task_dim[1], 1))
x2 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(low_input)
x2 = tf.keras.layers.Flatten()(x1)

ele_input = tf.keras.Input(shape=(task_dim[0], task_dim[1], 1))
x3 = tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(ele_input)
x3 = tf.keras.layers.Flatten()(x3)

other_input =  tf.keras.Input(shape=(3))
x4 = tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU())(other_input)

x = tf.keras.layers.Concatenate(axis=1)([x1, x2, x3, x4])
#x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(n_pred*np.prod(task_dim), activation=mapping_to_target_range)(x)
x = tf.keras.layers.Reshape([n_pred, task_dim[0], task_dim[1]])(x)
generator = tf.keras.Model([high_input, low_input, ele_input, other_input], x)
opt = tf.keras.optimizers.Adam(learning_rate=0.005)
generator.compile(optimizer=opt, loss='mean_absolute_error')

In [29]:
# define callbacks
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
best_save = tf.keras.callbacks.ModelCheckpoint('s2s_model', save_best_only=True, monitor='val_loss', mode='min')
callbacks = [lr_scheduler, early_stopping, best_save]

history = generator.fit([X_high[:-3], X_low[:-3], X_ele[:-3], X_other[:-3]], Y[:-3], epochs=1, callbacks=callbacks, validation_split=0.25)

4272/4273 [============================>.] - ETA: 0s - loss: 0.0072

INFO:tensorflow:Assets written to: s2s_model\assets


INFO:tensorflow:Assets written to: s2s_model\assets


4273/4273 [==============================] - 473s 109ms/step - loss: 0.0072 - val_loss: 0.0042 - lr: 0.0050


In [30]:
generator.predict([X_high[-2:], X_low[-2:], X_ele[-2:], X_other[-2:]])

array([[[[0.94876343, 0.95047736, 0.9487841 , 0.9495162 , 0.9492777 ,
          0.949939  , 0.94945574, 0.9492488 ],
         [0.9503256 , 0.9496294 , 0.9499347 , 0.949669  , 0.94927204,
          0.9499525 , 0.94992226, 0.9490282 ],
         [0.95024276, 0.95031774, 0.9495506 , 0.94941306, 0.9496045 ,
          0.9496205 , 0.9493502 , 0.94968736],
         [0.95150715, 0.95094347, 0.9487679 , 0.94924617, 0.9494501 ,
          0.9497068 , 0.94914925, 0.9506289 ],
         [0.94904065, 0.9499376 , 0.94915843, 0.94940746, 0.94950837,
          0.94953203, 0.94943166, 0.9494405 ],
         [0.9497731 , 0.9504872 , 0.9498053 , 0.9502927 , 0.9508985 ,
          0.949816  , 0.9488877 , 0.9508532 ],
         [0.9498416 , 0.9498662 , 0.950642  , 0.9499981 , 0.9507478 ,
          0.950603  , 0.9494418 , 0.94963557],
         [0.950281  , 0.9504169 , 0.9511137 , 0.950689  , 0.949062  ,
          0.94974154, 0.9501474 , 0.9495797 ]]],


       [[[0.9445611 , 0.94601655, 0.94508415, 0.9457435 , 0.

In [20]:
g2_model = tf.keras.models.load_model('s2s_model')

In [27]:
pd.DataFrame(history.history).to_csv('history.csv')

In [26]:
pred_Y = g2_model.predict([X_high[-1:], X_low[-1:], X_ele[-1:], X_other[-1:]])

In [28]:
pred_Y.mean()

0.9427106

In [112]:
np.sqrt(np.mean(np.square(pred_Y[-1] - Y[-1])))

0.010288388300887755

In [100]:
from util_tools.data_processing import rsquared
rsquared(pred_Y[0, 0].reshape((25)), Y[0,0].reshape((25)))

(0.008515162367835552, 0.6608753429840621)

In [13]:
pred_input = tf.keras.Input(shape=(n_pred, task_dim[0], task_dim[1]))
y1 = tf.keras.layers.Flatten()(pred_input)
'''
condition_input = tf.keras.Input(shape=(3))
y2 = tf.keras.layers.Dense(8, activation='relu')(condition_input)
y = tf.keras.layers.Concatenate(axis=1)([y1, y2])
'''
y = tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU())(y1)
y = tf.keras.layers.Dropout(0.8)(y)
y = tf.keras.layers.Dense(1, activation='sigmoid')(y)
discriminator = tf.keras.Model([pred_input], y)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
tf.__version__

'2.8.0'

In [14]:
#reload(util_tools.cGAN_model)
from util_tools.cGAN_model import  Condition_GAN

In [15]:
cGAN = Condition_GAN(generator, discriminator, lr=0.000001)
cGAN.fit(1, 100, [X_high, X_low, X_ele, X_other], Y)

Epoch:1, batch:1/1725, real_loss=1.043, fake_loss=1.745, g_loss=1.677
Epoch:1, batch:2/1725, real_loss=1.085, fake_loss=1.770, g_loss=1.734
Epoch:1, batch:3/1725, real_loss=0.466, fake_loss=2.828, g_loss=1.646
Epoch:1, batch:4/1725, real_loss=0.238, fake_loss=1.618, g_loss=1.587
Epoch:1, batch:5/1725, real_loss=0.971, fake_loss=0.732, g_loss=1.280
Epoch:1, batch:6/1725, real_loss=0.510, fake_loss=0.849, g_loss=1.648
Epoch:1, batch:7/1725, real_loss=0.519, fake_loss=0.571, g_loss=1.496
Epoch:1, batch:8/1725, real_loss=0.890, fake_loss=0.953, g_loss=1.634
Epoch:1, batch:9/1725, real_loss=1.193, fake_loss=1.387, g_loss=1.297
Epoch:1, batch:10/1725, real_loss=0.409, fake_loss=1.030, g_loss=1.364
Epoch:1, batch:11/1725, real_loss=0.618, fake_loss=1.761, g_loss=1.129
Epoch:1, batch:12/1725, real_loss=0.383, fake_loss=0.718, g_loss=1.037
Epoch:1, batch:13/1725, real_loss=1.119, fake_loss=1.130, g_loss=1.131
Epoch:1, batch:14/1725, real_loss=0.341, fake_loss=1.300, g_loss=0.901
Epoch:1, batch:

Epoch:1, batch:117/1725, real_loss=0.757, fake_loss=0.737, g_loss=0.702
Epoch:1, batch:118/1725, real_loss=0.684, fake_loss=0.697, g_loss=0.704
Epoch:1, batch:119/1725, real_loss=0.746, fake_loss=0.755, g_loss=0.699
Epoch:1, batch:120/1725, real_loss=0.662, fake_loss=0.676, g_loss=0.709
Epoch:1, batch:121/1725, real_loss=0.661, fake_loss=0.776, g_loss=0.709
Epoch:1, batch:122/1725, real_loss=0.696, fake_loss=0.684, g_loss=0.716
Epoch:1, batch:123/1725, real_loss=0.676, fake_loss=0.654, g_loss=0.721
Epoch:1, batch:124/1725, real_loss=0.715, fake_loss=0.666, g_loss=0.706
Epoch:1, batch:125/1725, real_loss=0.668, fake_loss=0.803, g_loss=0.719
Epoch:1, batch:126/1725, real_loss=0.691, fake_loss=0.780, g_loss=0.725
Epoch:1, batch:127/1725, real_loss=0.732, fake_loss=0.755, g_loss=0.717
Epoch:1, batch:128/1725, real_loss=0.719, fake_loss=0.755, g_loss=0.700
Epoch:1, batch:129/1725, real_loss=0.669, fake_loss=0.687, g_loss=0.709
Epoch:1, batch:130/1725, real_loss=0.629, fake_loss=0.755, g_los

Epoch:1, batch:231/1725, real_loss=0.744, fake_loss=0.692, g_loss=0.641
Epoch:1, batch:232/1725, real_loss=0.756, fake_loss=0.661, g_loss=0.654
Epoch:1, batch:233/1725, real_loss=0.777, fake_loss=0.662, g_loss=0.674
Epoch:1, batch:234/1725, real_loss=0.752, fake_loss=0.651, g_loss=0.675
Epoch:1, batch:235/1725, real_loss=0.703, fake_loss=0.725, g_loss=0.684
Epoch:1, batch:236/1725, real_loss=0.698, fake_loss=0.693, g_loss=0.702
Epoch:1, batch:237/1725, real_loss=0.709, fake_loss=0.635, g_loss=0.687
Epoch:1, batch:238/1725, real_loss=0.707, fake_loss=0.704, g_loss=0.696
Epoch:1, batch:239/1725, real_loss=0.665, fake_loss=0.729, g_loss=0.690
Epoch:1, batch:240/1725, real_loss=0.684, fake_loss=0.714, g_loss=0.706
Epoch:1, batch:241/1725, real_loss=0.714, fake_loss=0.825, g_loss=0.712
Epoch:1, batch:242/1725, real_loss=0.743, fake_loss=0.712, g_loss=0.712
Epoch:1, batch:243/1725, real_loss=0.710, fake_loss=0.682, g_loss=0.719
Epoch:1, batch:244/1725, real_loss=0.696, fake_loss=0.706, g_los

Epoch:1, batch:345/1725, real_loss=0.645, fake_loss=0.724, g_loss=0.695
Epoch:1, batch:346/1725, real_loss=0.688, fake_loss=0.708, g_loss=0.680
Epoch:1, batch:347/1725, real_loss=0.623, fake_loss=0.692, g_loss=0.682
Epoch:1, batch:348/1725, real_loss=0.685, fake_loss=0.681, g_loss=0.678
Epoch:1, batch:349/1725, real_loss=0.721, fake_loss=0.748, g_loss=0.684
Epoch:1, batch:350/1725, real_loss=0.673, fake_loss=0.690, g_loss=0.695
Epoch:1, batch:351/1725, real_loss=0.773, fake_loss=0.606, g_loss=0.685
Epoch:1, batch:352/1725, real_loss=0.700, fake_loss=0.687, g_loss=0.677
Epoch:1, batch:353/1725, real_loss=0.696, fake_loss=0.695, g_loss=0.683
Epoch:1, batch:354/1725, real_loss=0.733, fake_loss=0.684, g_loss=0.678
Epoch:1, batch:355/1725, real_loss=0.758, fake_loss=0.765, g_loss=0.701
Epoch:1, batch:356/1725, real_loss=0.674, fake_loss=0.756, g_loss=0.672
Epoch:1, batch:357/1725, real_loss=0.707, fake_loss=0.698, g_loss=0.692
Epoch:1, batch:358/1725, real_loss=0.706, fake_loss=0.693, g_los

Epoch:1, batch:459/1725, real_loss=0.670, fake_loss=1.082, g_loss=0.912
Epoch:1, batch:460/1725, real_loss=0.522, fake_loss=0.623, g_loss=0.878
Epoch:1, batch:461/1725, real_loss=0.663, fake_loss=0.880, g_loss=0.824
Epoch:1, batch:462/1725, real_loss=0.758, fake_loss=0.704, g_loss=0.771
Epoch:1, batch:463/1725, real_loss=0.601, fake_loss=0.775, g_loss=0.702
Epoch:1, batch:464/1725, real_loss=0.747, fake_loss=0.686, g_loss=0.690
Epoch:1, batch:465/1725, real_loss=0.721, fake_loss=0.691, g_loss=0.667
Epoch:1, batch:466/1725, real_loss=0.790, fake_loss=0.688, g_loss=0.654
Epoch:1, batch:467/1725, real_loss=0.701, fake_loss=0.666, g_loss=0.644
Epoch:1, batch:468/1725, real_loss=0.791, fake_loss=0.681, g_loss=0.662
Epoch:1, batch:469/1725, real_loss=0.740, fake_loss=0.627, g_loss=0.647
Epoch:1, batch:470/1725, real_loss=0.730, fake_loss=0.632, g_loss=0.643
Epoch:1, batch:471/1725, real_loss=0.738, fake_loss=0.685, g_loss=0.640
Epoch:1, batch:472/1725, real_loss=0.855, fake_loss=0.563, g_los

...

Epoch:1, batch:1255/1725, real_loss=0.624, fake_loss=0.651, g_loss=0.730
Epoch:1, batch:1256/1725, real_loss=0.660, fake_loss=0.778, g_loss=0.725
Epoch:1, batch:1257/1725, real_loss=0.692, fake_loss=0.722, g_loss=0.716
Epoch:1, batch:1258/1725, real_loss=0.628, fake_loss=0.737, g_loss=0.717
Epoch:1, batch:1259/1725, real_loss=0.627, fake_loss=0.705, g_loss=0.703
Epoch:1, batch:1260/1725, real_loss=0.682, fake_loss=0.743, g_loss=0.717
Epoch:1, batch:1261/1725, real_loss=0.632, fake_loss=0.680, g_loss=0.708
Epoch:1, batch:1262/1725, real_loss=0.720, fake_loss=0.699, g_loss=0.733
Epoch:1, batch:1263/1725, real_loss=0.653, fake_loss=0.723, g_loss=0.739
Epoch:1, batch:1264/1725, real_loss=0.626, fake_loss=0.784, g_loss=0.727
Epoch:1, batch:1265/1725, real_loss=0.686, fake_loss=0.747, g_loss=0.744
Epoch:1, batch:1266/1725, real_loss=0.638, fake_loss=0.763, g_loss=0.741
Epoch:1, batch:1267/1725, real_loss=0.711, fake_loss=0.719, g_loss=0.736
Epoch:1, batch:1268/1725, real_loss=0.618, fake_los

Epoch:1, batch:1368/1725, real_loss=0.720, fake_loss=0.668, g_loss=0.670
Epoch:1, batch:1369/1725, real_loss=0.697, fake_loss=0.739, g_loss=0.666
Epoch:1, batch:1370/1725, real_loss=0.723, fake_loss=0.699, g_loss=0.672
Epoch:1, batch:1371/1725, real_loss=0.708, fake_loss=0.687, g_loss=0.678
Epoch:1, batch:1372/1725, real_loss=0.645, fake_loss=0.714, g_loss=0.675
Epoch:1, batch:1373/1725, real_loss=0.741, fake_loss=0.682, g_loss=0.668
Epoch:1, batch:1374/1725, real_loss=0.741, fake_loss=0.620, g_loss=0.682
Epoch:1, batch:1375/1725, real_loss=0.676, fake_loss=0.679, g_loss=0.671
Epoch:1, batch:1376/1725, real_loss=0.698, fake_loss=0.631, g_loss=0.671
Epoch:1, batch:1377/1725, real_loss=0.726, fake_loss=0.671, g_loss=0.668
Epoch:1, batch:1378/1725, real_loss=0.757, fake_loss=0.701, g_loss=0.670
Epoch:1, batch:1379/1725, real_loss=0.772, fake_loss=0.702, g_loss=0.660
Epoch:1, batch:1380/1725, real_loss=0.681, fake_loss=0.673, g_loss=0.687
Epoch:1, batch:1381/1725, real_loss=0.666, fake_los

Epoch:1, batch:1481/1725, real_loss=0.674, fake_loss=0.792, g_loss=0.732
Epoch:1, batch:1482/1725, real_loss=0.668, fake_loss=0.710, g_loss=0.733
Epoch:1, batch:1483/1725, real_loss=0.602, fake_loss=0.724, g_loss=0.725
Epoch:1, batch:1484/1725, real_loss=0.639, fake_loss=0.799, g_loss=0.742
Epoch:1, batch:1485/1725, real_loss=0.653, fake_loss=0.776, g_loss=0.734
Epoch:1, batch:1486/1725, real_loss=0.635, fake_loss=0.726, g_loss=0.741
Epoch:1, batch:1487/1725, real_loss=0.630, fake_loss=0.690, g_loss=0.737
Epoch:1, batch:1488/1725, real_loss=0.654, fake_loss=0.786, g_loss=0.721
Epoch:1, batch:1489/1725, real_loss=0.703, fake_loss=0.754, g_loss=0.720
Epoch:1, batch:1490/1725, real_loss=0.686, fake_loss=0.763, g_loss=0.712
Epoch:1, batch:1491/1725, real_loss=0.681, fake_loss=0.699, g_loss=0.708
Epoch:1, batch:1492/1725, real_loss=0.680, fake_loss=0.732, g_loss=0.705
Epoch:1, batch:1493/1725, real_loss=0.662, fake_loss=0.708, g_loss=0.704
Epoch:1, batch:1494/1725, real_loss=0.720, fake_los

Epoch:1, batch:1594/1725, real_loss=0.688, fake_loss=0.656, g_loss=0.677
Epoch:1, batch:1595/1725, real_loss=0.709, fake_loss=0.656, g_loss=0.672
Epoch:1, batch:1596/1725, real_loss=0.699, fake_loss=0.690, g_loss=0.662
Epoch:1, batch:1597/1725, real_loss=0.712, fake_loss=0.656, g_loss=0.663
Epoch:1, batch:1598/1725, real_loss=0.713, fake_loss=0.685, g_loss=0.664
Epoch:1, batch:1599/1725, real_loss=0.709, fake_loss=0.681, g_loss=0.661
Epoch:1, batch:1600/1725, real_loss=0.737, fake_loss=0.668, g_loss=0.670
Epoch:1, batch:1601/1725, real_loss=0.696, fake_loss=0.675, g_loss=0.663
Epoch:1, batch:1602/1725, real_loss=0.679, fake_loss=0.699, g_loss=0.668
Epoch:1, batch:1603/1725, real_loss=0.718, fake_loss=0.663, g_loss=0.675
Epoch:1, batch:1604/1725, real_loss=0.692, fake_loss=0.680, g_loss=0.673
Epoch:1, batch:1605/1725, real_loss=0.720, fake_loss=0.659, g_loss=0.671
Epoch:1, batch:1606/1725, real_loss=0.726, fake_loss=0.676, g_loss=0.672
Epoch:1, batch:1607/1725, real_loss=0.706, fake_los

Epoch:1, batch:1707/1725, real_loss=0.608, fake_loss=0.816, g_loss=0.721
Epoch:1, batch:1708/1725, real_loss=0.650, fake_loss=0.744, g_loss=0.709
Epoch:1, batch:1709/1725, real_loss=0.642, fake_loss=0.665, g_loss=0.753
Epoch:1, batch:1710/1725, real_loss=0.697, fake_loss=0.750, g_loss=0.774
Epoch:1, batch:1711/1725, real_loss=0.632, fake_loss=0.749, g_loss=0.754
Epoch:1, batch:1712/1725, real_loss=0.681, fake_loss=0.766, g_loss=0.795
Epoch:1, batch:1713/1725, real_loss=0.684, fake_loss=0.681, g_loss=0.773
Epoch:1, batch:1714/1725, real_loss=0.632, fake_loss=0.837, g_loss=0.734
Epoch:1, batch:1715/1725, real_loss=0.722, fake_loss=0.746, g_loss=0.722
Epoch:1, batch:1716/1725, real_loss=0.618, fake_loss=0.691, g_loss=0.687
Epoch:1, batch:1717/1725, real_loss=0.709, fake_loss=0.701, g_loss=0.688
Epoch:1, batch:1718/1725, real_loss=0.712, fake_loss=0.661, g_loss=0.671
Epoch:1, batch:1719/1725, real_loss=0.680, fake_loss=0.670, g_loss=0.688
Epoch:1, batch:1720/1725, real_loss=0.716, fake_los

In [16]:
generator.predict([X_high[-1:], X_low[-1:], X_ele[-1:], X_other[-1:]])

array([[[[0.9444186 , 0.94252497, 0.945022  , 0.9438745 , 0.942726  ,
          0.9396411 , 0.94276845, 0.94287324],
         [0.9426934 , 0.9438174 , 0.9441983 , 0.94156   , 0.9439764 ,
          0.9411031 , 0.9446106 , 0.94374216],
         [0.9453738 , 0.9447421 , 0.946207  , 0.9442915 , 0.94202465,
          0.94314283, 0.9417516 , 0.9419898 ],
         [0.9424833 , 0.9421123 , 0.94184184, 0.94218767, 0.9434577 ,
          0.9420583 , 0.94411516, 0.9422333 ],
         [0.93986386, 0.93783283, 0.9448972 , 0.9446912 , 0.9411396 ,
          0.9392948 , 0.94373876, 0.9447304 ],
         [0.9397973 , 0.94336593, 0.94066215, 0.94264865, 0.9413771 ,
          0.9440998 , 0.94351906, 0.9419327 ],
         [0.9416085 , 0.94398415, 0.9430093 , 0.940447  , 0.93986964,
          0.9461374 , 0.9390241 , 0.94170934],
         [0.93982977, 0.94206476, 0.9439024 , 0.9414226 , 0.9441525 ,
          0.9456203 , 0.94261575, 0.94161844]]]], dtype=float32)

In [25]:
X = [X_high, X_low, X_ele, X_other]
j =0
batch_size=2
batch_X = [d[j*batch_size:(j+1)*batch_size] for d in X]

In [28]:
batch_X[0].shape

(2, 4, 5, 5, 1)

In [30]:
np.zeros((batch_size, 1), dtype='int')

array([[0],
       [0]])

# 3. downscaler test

In [17]:
#reload(downscale)
from util_tools import downscale

In [18]:
dscler = downscale.downscaler(generator)

In [19]:
h_data = g_data[10:10+n_lag]
l_data = m_data[1836: 1856]
days = list(range(1836, 1856))

In [20]:
downscaled_data = dscler.downscale(h_data, l_data, ele_data,  [G_lats, G_lons, M_lats, M_lons], days, n_lag, n_pred, task_dim)

In [21]:
downscaled_data.shape

(5, 123, 207)

In [22]:
h_data.shape

(15, 123, 207)

In [25]:
downscaled_data[4].mean()

0.934871974800356

In [26]:
h_data[10+4].mean()

0.9450469764791418